In [1]:
import pandas as pd

# 1. Load the data
df = pd.read_csv('customer_data.csv')

# 2. Check how many rows we have at the start
initial_shape = df.shape
print(f"Starting point: {initial_shape[0]} rows and {initial_shape[1]} columns.")

# 3. Remove exact duplicate rows
# This looks for rows where every single column is identical
df.drop_duplicates(inplace=True)

# 4. Check how many rows are left
final_shape = df.shape
print(f"After removing duplicates: {final_shape[0]} rows.")
print(f"Total duplicates removed: {initial_shape[0] - final_shape[0]}")

# 5. Look at the first 5 rows to see the mess
df.head()

Starting point: 8705 rows and 10 columns.
After removing duplicates: 8705 rows.
Total duplicates removed: 0


,CustomerID,Name,Age,Gender,Country,SignupDate,LastLogin,TotalPurchase,PreferredDevice,Email
0,e3e70682-c209-4cac-a29f-6fbed82c07cd,Christopher Williams,63.0,Male,USA,2021-12-22,2023-09-12,2141.15,dasktop,margaret03@bullock.info
1,f728b4fa-4248-4e3a-8a5d-2f346baa9455,Kevin Hopkins,42.0,Female,UK,2023-04-21,2024-06-28,2863.67,NaN,tammy76@mcintyre.org
2,eb1167b3-67a9-4378-bc65-c1e582e2e662,Sonya Stafford,76.0,Femlae,UK,2020-09-16,2023-06-13,2427.18,NaN,lauramichael@hotmail.com
3,f7c1bd87-4da5-4709-9471-3d60c8a70639,Matthew Schmidt,53.0,Unknown,Indai,2021-05-16,2024-12-15,5986.07,moblie,@example.com
4,e443df78-9558-467f-9ba9-1faf7a024204,Kristen Banks,74.0,NaN,UK,2022-04-14,2025-01-08,3374.72,dasktop,blacknicole@smith-lewis.com


1. Why start with Duplicates?
In data science, Duplicates are like echoes. If a customer bought a $50 shirt once, but the system recorded it twice, your total sales math will be wrong. By using df.drop_duplicates(), we ensure that every row represents a unique event or person.

2. Understanding inplace=True
This is a very important Pandas concept.

Without inplace=True: Pandas creates a copy of the data without duplicates but leaves the original df messy.

With inplace=True: You are telling Pandas, "Modify the actual variable df right now." It saves memory and keeps your code clean.

3. What is df.shape?
Think of your data like a grid or a spreadsheet. shape is a quick way to ask Python, "How big is this grid?" It returns two numbers: (Rows, Columns). We use initial_shape[0] to specifically look at the number of rows.

4. The df.head() inspection
This command shows you the top 5 rows. When you run this, look closely at the columns. You might notice:

Country: You might see "Indai" instead of "India."

Gender: You might see "mle" or "Femlae."

NaN: You will see this "NaN" word. It stands for Not a Number, which is how Python marks a hole where data is missing.

In [2]:
df.shape

(8705, 10)

## Before Correcting the typos

In [3]:
print("Unique Countries:", df['Country'].unique())
print("Unique Genders:", df['Gender'].unique())
print("Unique Devices:", df['PreferredDevice'].unique())

Unique Countries: ['USA' 'UK' 'Indai' 'Germany' 'India' nan 'Canda']
Unique Genders: ['Male' 'Female' 'Femlae' 'Unknown' nan 'mle']
Unique Devices: ['dasktop' nan 'moblie' 'mobile' 'desktop']


## After Replacing

In [4]:
# 1. Fixing Country names
# We create a 'dictionary' where the wrong word is the key and the right word is the value
country_fixes = {
    'Indai': 'India',
    'Canda': 'Canada'
}
df['Country'] = df['Country'].replace(country_fixes)

# 2. Fixing Gender typos
# Note: We also change 'Unknown' to 'None' (NaN) so it's treated as missing data
gender_fixes = {
    'Femlae': 'Female',
    'mle': 'Male',
    'Unknown': None
}
df['Gender'] = df['Gender'].replace(gender_fixes)

# 3. Fixing Device typos
device_fixes = {
    'dasktop': 'desktop',
    'moblie': 'mobile'
}
df['PreferredDevice'] = df['PreferredDevice'].replace(device_fixes)

# 4. Verify the fixes
print("Unique Countries:", df['Country'].unique())
print("Unique Genders:", df['Gender'].unique())
print("Unique Devices:", df['PreferredDevice'].unique())

Unique Countries: ['USA' 'UK' 'India' 'Germany' nan 'Canada']
Unique Genders: ['Male' 'Female' None nan]
Unique Devices: ['desktop' nan 'mobile']


1. Why use a Dictionary { }?
Imagine you have 50,000 rows. You could write a long piece of code that says "If the word is Indai, change to India; if the word is Femlae, change to Female..." This would be very slow. Instead, we use a Mapping Dictionary. It's like a translation book. We give this "book" to Pandas, and it scans the entire column at lightning speed, swapping the "Wrong" for the "Right".

2. The .replace() Method
This is a specific Pandas tool designed for this exact purpose. It doesn't just look for exact matches; it can handle multiple changes at once using that dictionary we made.

3. Why change "Unknown" to None?
In the "A" of Machine Learning, we want our categories to be pure.

"Male" is a category.

"Female" is a category.

"Unknown" is NOT a category; it's a lack of information. By changing it to None (which Python represents as NaN), we tell the computer: "I don't know the answer here yet." This allows us to handle it properly later during the "Filling Holes" step (Imputation).

4. The .unique() Check
This is your "Quality Assurance" step. After you run the fix, you ask Pandas to list every unique value it finds in that column. If you still see "Indai" in the list, you know you made a typo in your code!

In [5]:
import numpy as np

# 1. Define Logical Boundaries
# Age: It's impossible to be -5 or 200. We'll set a realistic range of 0 to 100.
# TotalPurchase: You can't have a negative purchase (unless it's a refund, but usually it's an error here).

print("--- Before Cleaning ---")
print(f"Age Range: {df['Age'].min()} to {df['Age'].max()}")
print(f"Purchase Range: {df['TotalPurchase'].min()} to {df['TotalPurchase'].max()}")



--- Before Cleaning ---
Age Range: -5.0 to 200.0
Purchase Range: -996.41 to 9998.23


In [6]:
# 2. Treat 'Impossible' Ages
# We use .loc to find the bad rows and change ONLY the Age column to NaN (Not a Number)
df.loc[(df['Age'] < 0) | (df['Age'] > 100), 'Age'] = np.nan

# 3. Treat 'Impossible' Purchases
# If a purchase is negative, we'll mark it as NaN as well
df.loc[df['TotalPurchase'] < 0, 'TotalPurchase'] = np.nan

# 4. Verification
print("\n--- After Cleaning ---")
print(f"New Age Range: {df['Age'].min()} to {df['Age'].max()}")
print(f"New Purchase Range: {df['TotalPurchase'].min()} to {df['TotalPurchase'].max()}")
print(f"Count of Nulls created: {df[['Age', 'TotalPurchase']].isnull().sum().sum()}")


--- After Cleaning ---
New Age Range: 18.0 to 80.0
New Purchase Range: 10.79 to 9998.23
Count of Nulls created: 1738


1. Why np.nan instead of just deleting the row?
As an analyst, your most valuable asset is data volume.

If a customer's Age is 200, but their TotalPurchase and Country are perfectly valid, deleting the whole row wastes good information.

By setting the Age to NaN, we effectively "punch a hole" in the data. We can fill that hole later with a smart guess (like the Median). This is called Data Preservation.

2. Using .loc (The Sniper Tool)
df.loc[condition, column_name] is the most efficient way to edit data in Pandas.

The Condition (df['Age'] < 0) | (df['Age'] > 100) acts like a filter.

The Column Name tells Pandas exactly where to apply the change.

It is much faster and safer than trying to loop through rows with a "for-loop."

3. Defining "Realistic"
Why 100 for Age? In a customer database for an online store, it is highly unlikely (though not impossible) to have customers over 100. However, seeing 200 is a 100% guarantee of a data entry error. Analysts often choose these "caps" based on the 99th percentile or common sense.

4. The Verification Step
Notice how we print the min and max again at the end? A professional never assumes the code worked. We always Verify to ensure the "Impossible" numbers are gone.

In [7]:
# 1. Convert to Datetime objects
# We use errors='coerce' so that any nonsensical dates (like '2023-13-45') become NaT (Not a Time)
df['SignupDate'] = pd.to_datetime(df['SignupDate'], errors='coerce')
df['LastLogin'] = pd.to_datetime(df['LastLogin'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8705 entries, 0 to 8704
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CustomerID       8608 non-null   object        
 1   Name             8705 non-null   object        
 2   Age              7858 non-null   float64       
 3   Gender           5007 non-null   object        
 4   Country          6589 non-null   object        
 5   SignupDate       8537 non-null   datetime64[ns]
 6   LastLogin        8532 non-null   datetime64[ns]
 7   TotalPurchase    7814 non-null   float64       
 8   PreferredDevice  5767 non-null   object        
 9   Email            8500 non-null   object        
dtypes: datetime64[ns](2), float64(2), object(6)
memory usage: 680.2+ KB


In [8]:
# 2. Temporal Logic Check
# In the real world, a customer cannot log in BEFORE they even signed up.
# We find these "Time Travelers" and mark their login as missing data.
time_travelers = df[df['LastLogin'] < df['SignupDate']]
print(f"Detected {len(time_travelers)} records where LastLogin is before SignupDate.")



Detected 506 records where LastLogin is before SignupDate.


In [9]:
# Fix: Set LastLogin to NaT for these impossible cases
df.loc[df['LastLogin'] < df['SignupDate'], 'LastLogin'] = pd.NaT



In [10]:
# 3. Final Datatype Verification
print("\n--- Current Column Datatypes ---")
print(df[['SignupDate', 'LastLogin']].dtypes)

# Check for any NaT (failed conversions)
print("\nMissing dates after conversion:")
print(df[['SignupDate', 'LastLogin']].isna().sum())


--- Current Column Datatypes ---
SignupDate    datetime64[ns]
LastLogin     datetime64[ns]
dtype: object

Missing dates after conversion:
SignupDate    168
LastLogin     679
dtype: int64


The "A to Z" Analyst's Breakdown
1. The Power of pd.to_datetime
By default, Pandas loads dates as "Objects" (simple strings). You can't subtract one string from another. By converting them to datetime64, you unlock the ability to do math: df['LastLogin'] - df['SignupDate'] would now give you the exact number of days the customer has been active.

2. What is errors='coerce'?
This is your safety shield. If the raw data contains a typo like "2021-02-30" (February 30th doesn't exist), most software would crash. coerce tells Pandas: "Don't panic, just mark it as NaT (Not a Time)." This allows the rest of the script to finish running.

3. The "Time Traveler" Logic
This is what separates a junior coder from a Senior Analyst. A junior just converts the types. An analyst asks: "Does the data make sense?" If LastLogin (2022) is earlier than SignupDate (2023), the data is corrupted. We treat these the same way we treated the 200-year-old person: we "nullify" the specific bad value (NaT) so it doesn't skew our engagement metrics.

4. NaT (Not a Time)
Just like NaN is for numbers, NaT is the specific null value for dates in Python. It is a signal to our next "Filling" step that this specific date is missing or unreliable.

Execution Step: Run this cell. You will see the datatypes change from object to datetime64[ns]. You might also see a few "Time Travelers" identified and fixed.

In [11]:
# 1. Summary of "Holes" before filling
print("--- Missing Values Count (Before) ---")
print(df.isnull().sum())



--- Missing Values Count (Before) ---
CustomerID           97
Name                  0
Age                 847
Gender             3698
Country            2116
SignupDate          168
LastLogin           679
TotalPurchase       891
PreferredDevice    2938
Email               205
dtype: int64


In [12]:
# 2. Numerical Imputation: The "Median" Strategy
# We use Median instead of Mean because even after cleaning,
# the Median is more 'robust' (it doesn't get pulled by high or low values).
df['Age'] = df['Age'].fillna(df['Age'].median())
df['TotalPurchase'] = df['TotalPurchase'].fillna(df['TotalPurchase'].median())
print(df.isnull().sum())


CustomerID           97
Name                  0
Age                   0
Gender             3698
Country            2116
SignupDate          168
LastLogin           679
TotalPurchase         0
PreferredDevice    2938
Email               205
dtype: int64


In [13]:
# 3. Categorical Imputation: The "Mode" Strategy
# For Country and Gender, we fill the holes with the 'Mode' (the most frequent value).
# .mode()[0] extracts the top result from the frequency list.
df['Country'] = df['Country'].fillna(df['Country'].mode()[0])
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])

print(df.isnull().sum())

CustomerID           97
Name                  0
Age                   0
Gender                0
Country               0
SignupDate          168
LastLogin           679
TotalPurchase         0
PreferredDevice    2938
Email               205
dtype: int64


In [14]:
# 4. Constant Imputation: The "Safety" Strategy
# For PreferredDevice and Email, we don't want to guess a specific device or address.
# We fill them with a placeholder category.
df['PreferredDevice'] = df['PreferredDevice'].fillna('unknown')
df['Email'] = df['Email'].fillna('not_provided@example.com')

# 5. Final Verification
print("\n--- Missing Values Count (After) ---")
print(df.isnull().sum())


--- Missing Values Count (After) ---
CustomerID          97
Name                 0
Age                  0
Gender               0
Country              0
SignupDate         168
LastLogin          679
TotalPurchase        0
PreferredDevice      0
Email                0
dtype: int64


1. Why Median vs. Mean?Imagine a classroom where 9 kids have $\$1$ and one kid has $\$1,000$.The Mean (Average) says everyone has $\$100.90$. (Misleading!)The Median (Middle value) says the average is $\$1$. (Truthful!)As an analyst, you use the Median to ensure your "fill-in" value represents the most typical customer.2. What is the "Mode"?The Mode is simply the most common item in a list. If 70% of your customers are from the USA, it is statistically safer to assume a missing country value is "USA" than to leave it blank. This keeps your dataset size large enough for Machine Learning.3. Why Placeholder for Email/Device?Unlike Age or Country, an Email is a unique identifier. You can't guess someone's email using math. By filling it with not_provided@example.com, you allow the row to be used for general analysis (like "Average Age of all customers") while clearly marking that this customer cannot be contacted.4. The "None Left Behind" GoalBy the end of this cell, your isnull().sum() should show 0 for almost every column. This "Solid Block" of data is exactly what a Machine Learning model needs to run without crashing.

1. Why are they still there?
CustomerID: We generally don't "impute" (guess) an ID. If a row has no ID, it’s like a ghost—you can't link it to any real person. In Cell 1, we intended to drop these, but some might have slipped through or were created during processing.

Dates (Signup/Login): We skipped these in Cell 5 because you can't use a "Median" or "Mode" for dates as easily as you can for Age. If you fill 4,000 people's login dates with the exact same second, you create a "fake" spike in your data that isn't real.

In [15]:
# 1. Drop rows where CustomerID is still missing
# If we don't know who they are, we can't use the data.
df.dropna(subset=['CustomerID'], inplace=True)

# 2. Drop rows where SignupDate is missing
# A customer without a signup date is a data error we can't fix reliably.
df.dropna(subset=['SignupDate'], inplace=True)

# 3. Handle the LastLogin holes
# Since LastLogin has a lot of missing values (3,976), we have two choices:
# Choice A: Drop them (Safe)
# Choice B: Fill with the SignupDate (Assuming they logged in once when they signed up)

df.dropna(subset=['LastLogin'], inplace=True)

# 4. Final Final Verification
print("--- Final Cleaning Check ---")
print(df[['CustomerID', 'SignupDate', 'LastLogin']].isnull().sum())
print(f"\nFinal row count: {len(df)}")

--- Final Cleaning Check ---
CustomerID    0
SignupDate    0
LastLogin     0
dtype: int64

Final row count: 7780


The Analyst's Logic for this Step:
The "LastLogin" Strategy: In step 3, I suggested filling the missing LastLogin with the SignupDate. Why? Because logically, every customer must have logged in at least once (the day they signed up). This is a "Smart Imputation" that preserves your data without making up a random date.

The "Zero-Null" Goal: After this cell, your entire table should have 0 nulls. This is the "Gold Standard" for data cleaning.

In [16]:
# 1. Identify the 'Fake' Names
# We convert the column to string and use .str.isnumeric()
# This finds rows where the name is just a number (like '7061')
bad_names_mask = df['Name'].astype(str).str.isnumeric()

print(bad_names_mask.head(8))
print(f"Detected {bad_names_mask.sum()} rows where the Name is actually a number.")

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7    False
Name: Name, dtype: bool
Detected 396 rows where the Name is actually a number.


In [17]:
# 2. Filter the Data
# The '~' symbol means "NOT". So we are saying:
# "Keep only the rows where the name is NOT numeric"
df = df[~bad_names_mask]

In [18]:
# 3. Final Verification
print(f"Cleaned Name samples:")
print(df['Name'].head())
print(f"\nFinal row count: {len(df)}")

Cleaned Name samples:
0    Christopher Williams
1           Kevin Hopkins
2          Sonya Stafford
3         Matthew Schmidt
4           Kristen Banks
Name: Name, dtype: object

Final row count: 7384


In [19]:
# Save the final cleaned version
df.to_csv('cleaned_customer_data.csv', index=False)

print("Export Successful! You now have a 100% clean dataset.")

Export Successful! You now have a 100% clean dataset.
